In [3]:
# Load deps for plotting, math, and UI

%matplotlib widget

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
from IPython.display import Audio
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pprint

matplotlib.rcParams['figure.figsize'] = [3, 2]

def start_figure(label, **kwargs):
    """Creates a figure and axes, closing an existing one"""
    plt.close(label)
    subplots = plt.subplots(**kwargs, num =label)
    plt.gcf().suptitle(label)
    return subplots


def fig_name():
    """Returns the name passed to start_figure"""
    return plt.gcf().get_label()

# Plotting pickup frequency response

A pickup in a circuit with active filters sees a resistive + capacitive input impedance as part of the buffer's input circuit. This modifies the frequency response of the pickup.

![pickup](http://buildyourguitar.com/resources/lemme/secrets3.gif)

With $C_L$ and $R_L$ the load, $Z_1$ the lumped impedance of the pickup inductance and resistance, $Z_2$ the lumped impedance of the pickup capacitance and load, $V_O$ the potential before the load, and $V_I$ the unfiltered pickup response,

$$
\frac{V_O}{V_I} = \frac{IZ_2}{I(Z_1 + Z_2)} = \frac{1}{(C + C_L)Ls^2 + (\frac{L}{R_L} + R(C + C_L))s + (1 + \frac{R}{R_L})}
$$

We can feed this into [scipy.signal.freqs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqs.html#scipy.signal.freqs) or scipy.signal.freqz. I'm not sure what the difference between the "analog" and "digital" variants is.

In [5]:
# Typical circuit values for a guitar
L = 3.0  # Henry
R = 7e3  # Ohm
C = 300e-12  # Farad
R_L = 1e6  # Ohm, from the cable and volume
C_L = 300e-12  # Farad, from the cable and tone


def pickup_frequency_response(f, r, l, c, r_l, c_l):
    """Return the complex pickup frequency response given circuit values"""

    # Numerator coefficients
    b = [1]
    # Denominator coefficients
    a = [(c + c_l) * l, (l / r_l + r * (c + c_l)), 1 + r / r_l]
    _, h = scipy.signal.freqs(b, a, worN=f * 2 * np.pi)

    return h


fig, ax = start_figure('Pickup Frequency Response', figsize=(6, 4))
ax.set_xlabel('Hz')
ax.set_ylabel('Gain')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(1e-1, 10)
ax.set_title(fig_name())

f = np.linspace(0, 20e3, 400)

response_line, = ax.plot(f, np.ones(f.shape))


@interact(r=widgets.FloatSlider(min=1e3,
                                max=20e3,
                                step=.5e3,
                                value=7e3,
                                description='R (Ohm)'),
          l=widgets.FloatSlider(min=.5,
                                max=7,
                                step=.2,
                                value=3,
                                description='L (H)'),
          c=widgets.FloatSlider(min=100,
                                max=700,
                                step=20,
                                value=300,
                                description='C (pF)'),
          r_l=widgets.FloatLogSlider(min=3,
                                     max=7,
                                     step=.1,
                                     value=1e6,
                                     description='R_L (Ohm)'),
          c_l=widgets.FloatSlider(min=50,
                                  max=2000,
                                  step=50,
                                  value=300,
                                  description='C_L (pF)'))
def update_frequency_response(r, l, c, r_l, c_l):
    h = pickup_frequency_response(f, r, l, c * 1e-12, r_l, c_l * 1e-12)
    response_line.set_ydata(abs(h))

FigureCanvasNbAgg()

TypeError: cannot unpack non-iterable NoneType object

In [6]:
def series(*Zs):
    return lambda s: sum(Z(s) for Z in Zs)


def parallel(*Zs):
    return lambda s: 1.0 / (sum(1.0 / Z(s) for Z in Zs))


def cap(capacitance):
    return lambda s: 1.0 / (capacitance * s)


def ind(inductance):
    return lambda s: s * inductance


def res(resistance):
    return lambda s: resistance


def pickup_volume_tone_frequency_response(f, r_pickup, l_pickup, c_pickup,
                                          c_tone, r_pot, volume_value,
                                          tone_value, r_load, c_load):
    """complex transfer function for a pickup with a volume and tone control."""

    Z_pickup = series(ind(l_pickup), res(r_pickup))
    Z_tone = parallel(cap(c_pickup), series(res(r_pot * tone_value),
                                            cap(c_tone)))
    Z_volume = res(r_pot * (1.0 - volume_value))
    Z_load = parallel(res(r_pot * volume_value), cap(c_load), res(r_load))

    alpha = lambda s: Z_volume(s) / Z_load(s) + 1.0

    H = lambda s: Z_pickup(s)**-1 / (alpha(s) * Z_pickup(s)**-1 + alpha(s) *
                                     Z_tone(s)**-1 + Z_load(s)**-1)

    return H(2 * np.pi * f * 1j)


In [8]:
# Typical circuit values for a guitar
L_Pickup = 3.0  # Henry
R_Pickup = 7e3  # Ohm
C_Pickup = 300e-12  # Farad
C_Tone = 22e-9  # Farad. For humbucker. Single coils use 47e-9
R_Pot = 250e3  # Ohm. For humbucker. Single coils use 500k
Volume_Value = 1  # Position of the volume knob, 1 is full volume
Tone_Value = 1  # Position of the tone knob, 1 is full tone
R_Load = 1e6  # Ohm, primarily from the amp
C_Load = 300e-12  # Farad, primarily from the cable

fig, ax = start_figure('Pickup + Volume + Tone Frequency Response',
                       figsize=(6, 4))
ax.set_xlabel('Hz')
ax.set_ylabel('Gain')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(1e-1, 10)
ax.set_title(fig_name())

f = np.linspace(20, 20e3, 400)

response_line, = ax.plot(f, np.ones(f.shape))


@interact(r_pickup=widgets.FloatSlider(min=1e3,
                                       max=20e3,
                                       step=.5e3,
                                       value=7e3,
                                       description='Pickup R (Ohm)'),
          l_pickup=widgets.FloatSlider(min=.5,
                                       max=7,
                                       step=.2,
                                       value=3,
                                       description='Pickup L (H)'),
          c_pickup=widgets.FloatSlider(min=100,
                                       max=700,
                                       step=20,
                                       value=300,
                                       description='Pickup C (pF)'),
          c_tone=widgets.FloatSlider(min=0,
                                     max=100,
                                     step=5,
                                     value=22,
                                     description='C (nF)'),
          v_value=widgets.FloatSlider(min=0,
                                      max=1,
                                      step=.02,
                                      value=1,
                                      description='Volume knob'),
          t_value=widgets.FloatSlider(min=0,
                                      max=1,
                                      step=.01,
                                      value=1,
                                      description='Tone knob'),
          r_load=widgets.FloatLogSlider(min=3,
                                        max=7,
                                        step=.1,
                                        value=1e6,
                                        description='Load R (Ohm)'),
          c_load=widgets.FloatSlider(min=50,
                                     max=2000,
                                     step=50,
                                     value=300,
                                     description='Load C (pF)'))
def update_frequency_response(r_pickup, l_pickup, c_pickup, c_tone, v_value,
                              t_value, r_load, c_load):

    h = pickup_volume_tone_frequency_response(f, r_pickup, l_pickup,
                                              c_pickup * 1e-12, c_tone * 1e-9,
                                              R_Pot, v_value, t_value, r_load,
                                              c_load * 1e-12)

    response_line.set_ydata(abs(h))


FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=7000.0, description='Pickup R (Ohm)', max=20000.0, min=1000.0, step=50…

# Psychoacoustic and Physical Guitar Tone Modeling

Guitar players use a lot of poetic terminology to describe sounds, like "crunchy", "tight", "flubby", "aggressive", "growl", etc. In the end, this is a language that guitarists use to describe sounds. The language is valuable because it provides a more expressive way to communicate about sounds. The language can also be manipulated by marketing, which can takes advantage of the loosely defined terminology to upsell products. 

Guitarists are always looking make the sound they imagine and describe in their language a reality. This is a hard, creative, emergent, non-deterministic, heuristic, and essential process. Bridging the gap between what you imagine and what is possible is what makes it music, and musicians will seek that out no matter how good technology gets. So making it easy to achieve the sounds you imagine frees our minds up to make more creative music.

# Problems

- Guitar electronics can be finiky
- Can be hard to dial in tone
- Volume issues

# Solutions

- Custom control inputs
- Digital control units
- Onboard preamps
- Digital connection between pedal and guitar